In [ ]:
#!/bin/env python3

import datetime
from pathlib import Path

import h5py
import numpy as np


def ALBA_STXM_to_MANTIS(path_file) -> None:
    # Read all needed dataset via keys
    with h5py.File(path_file, "r") as Fin:
        data = np.array(Fin["FastAligned/spec_aligned"]).astype("float32")
        energy = np.array(Fin["FastAligned/energy"])
        x_pixel_size = np.array(Fin["FastAligned/x_pixel_size"])
        y_pixel_size = np.array(Fin["FastAligned/y_pixel_size"])
        currents = np.array(Fin["FastAligned/Currents"]).astype("float16")
        exptimes = np.array(Fin["FastAligned/ExpTimes"])
        rotation_angle = np.array(Fin["FastAligned/rotation_angle"])

    ## rewirte it with NeXus format
    new_file_name = f"NX_{path_file.stem}"
    with h5py.File(f"{new_file_name}.hdf5", "w") as root:
        ## Create the GROUPS
        root.create_group("entry")
        root["/entry"].attrs["NX_class"] = "NXentry"
        root["/entry/"].create_group("sample")
        root["/entry/sample"].attrs["NX_class"] = "NXsample"
        root["/entry/"].create_group("data")
        root["/entry/data"].attrs["NX_class"] = "NXdata"
        root["/entry/"].create_group("control")
        root["/entry/control"].attrs["NX_class"] = "NXmonitor"

        ## Create the basic ATTRIBUTES
        root["/"].attrs["default"] = "entry"
        root["/entry"].attrs["default"] = "data"
        root.attrs["file_name"] = new_file_name
        root.attrs["file_time"] = datetime.datetime.now().isoformat()
        root.attrs["h5py_version"] = h5py.version.version
        root.attrs["HDF5_Version"] = h5py.version.hdf5_version
        root["/entry"].create_dataset(name="definition", data=[np.bytes_("NXstxm")])

        ## Set up the main data group
        root["/entry/data"].attrs["signal"] = np.bytes_("data")
        root["/entry/data"].attrs["axes"] = [np.bytes_("energy"), np.bytes_("sample_y"), np.bytes_("sample_x")]
        # root['/entry/data'].attrs['energy_indices'] = [0]
        # root['/entry/data'].attrs['sample_y_indices'] = [1]
        # root['/entry/data'].attrs['sample_x_indices'] = [2]
        root["/entry/data"].create_dataset(
            name="stxm_scan_type", data=[np.bytes_("sample image stack")], maxshape=None, compression="lzf"
        )

        root["/entry/data"].create_dataset(name="data", data=data, maxshape=None)
        root["/entry/data/data"].attrs["units"] = "mA"

        root["/entry/data"].create_dataset(name="energy", data=energy, maxshape=None)
        root["/entry/data/energy"].attrs["units"] = "eV"

        root["/entry/data"].create_dataset(
            name="sample_y", data=np.linspace(0, data.shape[1] * y_pixel_size, num=data.shape[1]), maxshape=None
        )
        root["/entry/data/sample_y"].attrs["units"] = "um"

        root["/entry/data"].create_dataset(
            name="sample_x", data=np.linspace(0, data.shape[2] * x_pixel_size, num=data.shape[2]), maxshape=None
        )
        root["/entry/data/sample_x"].attrs["units"] = "um"

        root["/entry/data"].create_dataset(name="count_time", data=exptimes, maxshape=None)
        root["/entry/data/count_time"].attrs["units"] = "ms"

        ## Set up the control data group for ring current normalisation
        root["/entry/control"].attrs["signal"] = np.bytes_("data")
        root["/entry/control"].attrs["axes"] = [np.bytes_("energy"), np.bytes_("sample_y"), np.bytes_("sample_x")]
        # root['/entry/control'].attrs['energy_indices'] = [0]
        # root['/entry/control'].attrs['sample_y_indices'] = [1]
        # root['/entry/control'].attrs['sample_x_indices'] = [2]

        root["/entry/control"].create_dataset(
            name="data",
            data=np.broadcast_to(np.moveaxis(np.atleast_3d(currents), 1, 0), data.shape),
            maxshape=None,
            compression="lzf",
        )
        # root['/entry/control/data'].attrs['units'] = 'mA'

        root["/entry/control"].create_dataset(name="energy", data=energy, maxshape=None)
        root["/entry/control/energy"].attrs["units"] = "eV"

        root["/entry/control"].create_dataset(
            name="sample_y", data=np.linspace(0, data.shape[1] * y_pixel_size, num=data.shape[1]), maxshape=None
        )
        root["/entry/control/sample_y"].attrs["units"] = "um"

        root["/entry/control"].create_dataset(
            name="sample_x", data=np.linspace(0, data.shape[2] * x_pixel_size, num=data.shape[2]), maxshape=None
        )
        root["/entry/control/sample_x"].attrs["units"] = "um"
        root["/entry/control"].create_dataset(name="count_time", data=exptimes, maxshape=None)
        root["/entry/control/count_time"].attrs["units"] = "ms"

        root["/entry/sample"].create_dataset(name="rotation_angle", data=rotation_angle, maxshape=None)
        root["/entry/sample/rotation_angle"].attrs["units"] = "degrees"

In [ ]:
path_folder = Path(r"C:\Users\chengliu\Desktop\20220611_caref_-28-3x294-6y_specnorm_ali-hdf5_2025-01-14_0850")
path_file = Path.joinpath(path_folder, r"20220611_CaRef_-28.3x294.6y_specnorm_ali.hdf5")
ALBA_STXM_to_MANTIS(path_file)